# HPE Redfish Ansible playbooks

Version 0.123

## Introduction

The goal of this notebook is to present another approach for creating Redfish Ansible Playbooks compared to the approach used in the previous notebooks.

The HPE Ansible Redfish [Gitub repository](https://github.com/HewlettPackard/ansible-ilorest-role) proposes three sets of Ansible Redfish Playbooks examples using:

  * the iLOrest command
  * the Redfish Galaxy collection
  * Ansible modules derived from the [examples](https://github.com/HewlettPackard/python-ilorest-library/tree/master/examples/Redfish) of the `python-ilorest-library`.

In this Jupyter Notebook, you will study an [Ansible Redfish playbook](3/set_uid_light.yml) based upon the [`set_uid_light.py`](https://github.com/HewlettPackard/python-ilorest-library/tree/master/examples/Redfish/set_uid_light.py) example of the the HPE [`python-ilorest-library`](https://github.com/HewlettPackard/python-ilorest-library/tree/master/). This python example has been modified to become an [Ansible module](https://github.com/HewlettPackard/ansible-ilorest-role/blob/master/library/set_uid_light.py). 

## <a id = "EnvironmentPreparation">Environment preparation</a>

The following cell sets environment variables and checks the connectivity toward the various BMCs used in this notebook.

In [ ]:
########## Environment preparation (Version: 0.132) ############

# Set Student ID number
stdid={{ STDID }}
Id=$(id --user --name)
NbId=3
InvFile=${NbId}/hosts

# location and ports variables
IloSyBasePort={{ ILOSYBASEPORT }}
let iLO5SimulatorBasePort=$IloSyBasePort
let iLO5SimulatorPort=${iLO5SimulatorBasePort}+${stdid}

iLO5SimulatorIP=ilo5simulators
iLOSimulator=${iLO5SimulatorIP}:${iLO5SimulatorPort}
iLO5SimulatorURI=https://${iLOSimulator}

# Fake Credentials as we are testing against a BMC simulator
OvSsoToken="FakeOvSsoToken"

# Miscellaneous
WorkshopDir=$PWD
HpePythonRedfishVenv="${NbId}/HpePythonRedfishVenv"
export PYTHONPATH="${WorkshopDir}/${NbId}/library/"
w=$(basename $PWD)

alias ResetSimulators="../create-globalbmc.shc.x &>/dev/null; sleep 1"

# Verify we can reach the remote Bmcs on the right HTTPS ports. 
for bmc in iLO5Simulator ; do
    ip="${bmc}IP" ; port=$(echo ${bmc}Port)
    nc -vz  $(eval echo "\$${ip}") $(eval echo "\$${port}") &>/dev/null && 
        echo -e "\n\tGood News: $bmc is reachable" \
        || echo "WARNING: Problem reaching $bmc"
done


# Create the Ansible inventory file
cat > ${InvFile} << __EOF__
[OneViewManagedBmcs]
${iLO5SimulatorIP} ansible_port=${iLO5SimulatorPort}

[OneViewManagedBmcs:vars]
ansible_python_interpreter=${WorkshopDir}/${HpePythonRedfishVenv}/bin/python3
ansible_search_path=${HpePythonRedfishVenv}

# Below is a fake session token as we are testing against an iLO 5 simulator.
# In real life, you should populate this variable with the token obtained
# lab 1 of this workshop.
token="${OvSsoToken}"
__EOF__


## Virtual Python environment creation

In order to completely isolate this notebook environment from other notebooks or student python environments, it is safer to create your dedicated Python virtual environment.

> NOTE: This `Venv` creation can take up to **2 minutes**. Just wait until message `Finished creating Venv` is diplayed

In [ ]:
# Create Virtual Python environment (Venv) [Version 0.111]
[ -d ${HpePythonRedfishVenv} ] && rm -r ${HpePythonRedfishVenv} &>/dev/null 
python3 -m venv ${HpePythonRedfishVenv}                         &>/dev/null
source ${HpePythonRedfishVenv}/bin/activate                     &>/dev/null
PS1="[PEXP\[\]ECT_PROMPT>"                                      # Avoid Venv long prompt messing up outputs

# Populate Python Venv with the HPE python-ilorest-library
pip install wheel                                               &>/dev/null
pip install python-ilorest-library                              &>/dev/null

# Install latest Ansible in the Venv
pip install jmespath                                            &>/dev/null
pip install ansible                                             &>/dev/null

echo -e "\n\n\tFinished creating Venv\n\n"

### <a id = "iLO5SimulatorRestart">Restart iLO 5 simulator</a>

If you need or desire to restart your iLO 5 simulator to restart this workshop from scratch or for other reasons, run the following cell at any time.

In [ ]:
# iLO 5 Simulator restart
ResetSimulators 

# Verify we can reach the remote Bmcs on the right HTTPS ports. 
for bmc in iLO5Simulator ; do
    ip="${bmc}IP" ; port=$(echo ${bmc}Port)
    nc -vz  $(eval echo "\$${ip}") $(eval echo "\$${port}") &> /dev/null && 
        echo "$bmc is reachable" \
        || echo "WARNING: Problem reaching $bmc"
done

## Get and set Redfish properties using HPE's python-ilorest-library

In the previous notebook, to discover the `Chassis` collection and the value of the indicator LEDs, you had to crawl the Redfish tree using Ansible and its built-in `uri` module.

When using a custom Ansible module derived from a `python-ilorest-library` example, the Redfish tree crawling is performed inside the Ansible module. 

>**NOTE**: TBD Examples are not bullet proofed, they are just a proof of concept./TBD

HPE `python-ilorest-library` has been loaded in the Python Venv of the second cell of this notebook. It will allow the creation of a redfish object containing authentication token as well as GET and SET methods.

You will find your custom Ansible modules and other python files in your `library` sub-folder.

### Indicator LED

TBD

In [ ]:
# Launch Ansible Playbook to toggle Indicator LED
ansible-playbook -i ${InvFile} ${NbId}/SetIndicatorLEDUsingiLOrestLibrary.yml

# ****  NEED To Decide wether the following is relevant in this workshop ****

**Pro**: the Security Dashboard parameters is an HPE/Oem extension. This example can show how chiant it is to crawl and discover all of them using the Ansible `uri`

**Cons**: May be too long for 105 minutes ? May be too long and too chiant to develop ?




## Example 2: GET and SET another TBD parameter

TBD

### Get Security params

The following cell retrieves the list of all the security parameters present in the security dashboard of the iLO 5 Graphical User Interface as shown on this picture:

<img src="Pictures/SecurityDashboard.png" alt="iLO 5 Security Dashboard" style="width: 700px;" align=middle />

To retrieve this list of security parameters and their properties, the [`get_security_params.yml`](3/get_security_params.yml) file calls the [`get_security_params.py`](3/library/get_security_params.py) Ansible module. This Ansible module has the same architecture as the examples published in the [HPE Redfish/Ansible](https://github.com/HewlettPackard/ansible-ilorest-role/tree/master/library) GitHub repository.

>Note: The architecture of these HPE Ansible/Redfish modules is derived from the examples published in the [python-ilorest-library](https://github.com/HewlettPackard/python-ilorest-library/tree/master/examples/Redfish) GitHub repository.

? do we need more explanation of this architecture and the Resource Directory flag ?


In [ ]:
ansible-playbook -i ${NbId}/hosts ${NbId}/get_security_params.yml

### Modify a security parameter property

In the output of the above cell, some security parameters have their `Ignore` property set to set to `False` or `True`..... TBD

The following cell runs the [`set_ignore_security_param`](3/set_ignore_security_param.yml) Ansible Playbook in order to TBD

In [ ]:
ansible-playbook -i ${NbId}/hosts ${NbId}/set_ignore_security_param.yml

ansible-playbook -i ${NbId}/hosts ${NbId}/get_security_params.yml

# ****  END of NEED To Decide wether the following is relevant in this workshop ****

## Summary

In this workshop, you TBD Go to the [Conclusion Notebook](3-Conclusion.ipynb) if you are finished.